In [1]:
%%capture
%pip install "unsloth[colab-new] @git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps xformers trl peft accelerate bitsandbytes
%pip install -q torch
%pip install -q llama-index
%pip install -q llama-index-llms-huggingface
%pip install -q llama-index-embeddings-huggingface
!pip install llama-index-vector-stores-chroma chromadb



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
import chromadb
import torch


In [ ]:
chroma_path = "/content/chroma_storage"
db = chromadb.PersistentClient(path=chroma_path)
chroma_collection = db.get_or_create_collection("dataset_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    device=device
)

Settings.embed_model = embed_model
Settings.transformations = [SentenceSplitter(chunk_size=1024)]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

input_csv = Path("/content/drive/MyDrive/Colab Notebooks/dataset/pp_recipes.csv")
split_dir = Path("/content/drive/MyDrive/Colab Notebooks/dataset/splits")
split_dir.mkdir(exist_ok=True)

chunksize = 80_000
for i, chunk in tqdm(enumerate(pd.read_csv(input_csv, chunksize=chunksize))):
    part_path = split_dir / f"part_{i}.csv"
    chunk.to_csv(part_path, index=False)


5it [01:06, 12.71s/it]/usr/local/lib/python3.12/dist-packages/tqdm/std.py:1181: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
7it [01:27, 12.44s/it]


In [ ]:
# from sentence_transformers import SentenceTransformer
# from chromadb.utils import embedding_functions
# import chromadb
# import pandas as pd

# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

# db = chromadb.PersistentClient(path="/content/chroma_storage_fast")
# collection = db.get_or_create_collection("dataset_collection")

# for csv_path in tqdm(list(split_dir.glob("*.csv"))):
#     df = pd.read_csv(csv_path)
#     texts = df.astype(str).apply(lambda row: " ".join(row.values), axis=1).tolist()

#     # Batch embed and insert
#     batch_size = 128 * 16
#     for i in range(0, len(texts), batch_size):
#         batch = texts[i:i+batch_size]
#         embeddings = model.encode(batch, batch_size=batch_size, show_progress_bar=False)
#         ids = [f"{csv_path.stem}_{i+j}" for j in range(len(batch))]
#         collection.add(documents=batch, embeddings=embeddings.tolist(), ids=ids)


 71%|███████▏  | 5/7 [45:28<18:44, 562.33s/it]/tmp/ipython-input-2875028538.py:12: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
100%|██████████| 7/7 [59:28<00:00, 509.85s/it]


In [5]:
from chromadb.utils import embedding_functions
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex

# 1️⃣ Connect to existing persistent Chroma DB
db = chromadb.PersistentClient(path="/content/drive/MyDrive/Colab Notebooks/chroma_storage_fast")

# 2️⃣ Load your existing collection
collection = db.get_or_create_collection("dataset_collection")

# 3️⃣ Wrap it for LlamaIndex
vector_store = ChromaVectorStore(chroma_collection=collection)

# 4️⃣ Create a storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 5️⃣ Build an index directly from your existing store
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

# 6️⃣ Create retriever
retriever = index.as_retriever(similarity_top_k=5)


In [ ]:
# from llama_index.vector_stores.chroma import ChromaVectorStore
# from llama_index.core import StorageContext, VectorStoreIndex

# vector_store = ChromaVectorStore(chroma_collection=collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_vector_store(vector_store)


In [6]:
# -------------- prerequisites --------------
# (run these once)
# !pip install chromadb llama-index sentence-transformers transformers  # if not already installed

# -------------- load your HF model (example) --------------
from unsloth import FastLanguageModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "azimidokht/recipe-recom-fine-tuned",
    max_seq_length = 2048,
    dtype = None,           # or "float16"/"bfloat16"
    load_in_4bit = True,    # as you used
)
# If your FastLanguageModel needs explicit device move, do it here:
# model.to(device)  # only if required by the class



/tmp/ipython-input-1159825941.py:6: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# retriever = index.as_retriever(similarity_top_k=5)


In [7]:
random_state=3407
prompt = """
### Human:
{}

### Assistant
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("azimidokht/recipe-recom", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

split_dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=random_state)

train_dataset = split_dataset["train"]
temp_dataset  = split_dataset["test"]

val_test_split = temp_dataset.train_test_split(test_size=0.5, shuffle=True, seed=random_state)

val_dataset  = val_test_split["train"]
test_dataset = val_test_split["test"]


recipe_recom.parquet:   0%|          | 0.00/214k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5149 [00:00<?, ? examples/s]

Map:   0%|          | 0/5149 [00:00<?, ? examples/s]

In [8]:
import random
from tqdm import tqdm



In [9]:
prompt = """
### Human:
{}
{}

### Assistant
{}"""


inferenced_data = {}

def generate_predictions(dataset, model, tokenizer, max_new_tokens=128):
    """Generate model outputs for a HF datasets split of dicts with keys: instruction, output."""
    # sampel to overcome the resource limit
    sample_size = 50
    indices = random.sample(range(len(dataset)), sample_size)
    sampled_dataset = dataset.select(indices)

    predictions, references, instructions = [], [], []
    FastLanguageModel.for_inference(model)
    for example in tqdm(sampled_dataset):
        instruction = example["instruction"]
        reference   = example["output"]

        # retrieve relevant context
        try:
          retrieved_nodes = retriever.retrieve(instruction)
          # Do something with retrieved_nodes
          print(f"\nRetrieved {len(retrieved_nodes)} nodes for: {instruction[:50]}...")

        except Exception as e:
          print(f"⚠️ Error retrieving for instruction: {instruction[:50]}... — {e}")
          continue
        context_texts = "\n\n".join([n.node.get_content() for n in retrieved_nodes])


        inputs = tokenizer(
            [prompt.format(instruction, context_texts, "")],
            return_tensors="pt"
        ).to("cuda")

        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # It Extracts just the assistant section
        # Falls back to full text if the delimiter isn't present
        #If the model output doesn’t include the separator (### Assistant), use the full text as the prediction instead of trying to split it.
        if "### Assistant" in generated_text:
            assistant_response = generated_text.split("### Assistant", 1)[-1].strip()
        else:
            assistant_response = generated_text.strip()

        predictions.append(assistant_response)
        references.append(reference)
        instructions.append(instruction)
    return predictions, references, instructions


# def evaluate(predictions, references, metrics: list[str]):
#     """
#     Evaluate the predictions against the references using the specified metrics.
#     """
#     results = {}
#     if "rouge" in metrics:
#       results["rouge"] = rouge.compute(predictions=predictions, references=references)
#     if "bleu" in metrics:
#       results["bleu"] = bleu.compute(predictions=predictions, references=[[ref] for ref in references]) # BLEU expects a list of references
#     else:
#       raise ValueError("Invalid metric")

#     return results

# def store(data, path):
#   with open(path, "w") as f:
#     json.dump(data, f)



In [10]:
test_predictions, test_references, test_inst = generate_predictions(test_dataset, model, tokenizer)

  0%|          | 0/50 [00:00<?, ?it/s]


Retrieved 0 nodes for: Suggest a ingredients recipe under 112 calories....


  2%|▏         | 1/50 [00:14<11:57, 14.65s/it]


Retrieved 0 nodes for: List a low-sodium recipe under 455mg sodium....


  4%|▍         | 2/50 [00:19<07:06,  8.89s/it]

...

 96%|█████████▌| 48/50 [02:28<00:03,  1.93s/it]


Retrieved 0 nodes for: Suggest a balanced meal with moderate calories and...


 98%|█████████▊| 49/50 [02:31<00:02,  2.44s/it]


Retrieved 0 nodes for: Suggest a balanced meal with moderate calories and...


100%|██████████| 50/50 [02:34<00:00,  3.10s/it]


In [11]:
val_predictions_base, val_references_base, val_inst_base = generate_predictions(val_dataset, model, tokenizer)

  0%|          | 0/50 [00:00<?, ?it/s]


Retrieved 0 nodes for: Recommend a quick recipe that takes less than 40 m...


  2%|▏         | 1/50 [00:01<01:30,  1.84s/it]


Retrieved 0 nodes for: Find a high-protein recipe with at least 17g prote...


  4%|▍         | 2/50 [00:04<02:03,  2.58s/it]

...

 96%|█████████▌| 48/50 [01:51<00:04,  2.29s/it]


Retrieved 0 nodes for: Recommend a quick recipe that takes less than 70 m...


 98%|█████████▊| 49/50 [01:52<00:02,  2.13s/it]


Retrieved 0 nodes for: Suggest a ingredients recipe around 689 calories....


100%|██████████| 50/50 [01:56<00:00,  2.32s/it]


In [13]:
import json
def evaluate(predictions, references, metrics: list[str]):
    """
    Evaluate the predictions against the references using the specified metrics.
    """
    results = {}
    if "rouge" in metrics:
      results["rouge"] = rouge.compute(predictions=predictions, references=references)
    if "bleu" in metrics:
      results["bleu"] = bleu.compute(predictions=predictions, references=[[ref] for ref in references]) # BLEU expects a list of references
    else:
      raise ValueError("Invalid metric")

    return results

def store(data, path):
  with open(path, "w") as f:
    json.dump(data, f)

In [14]:
inferenced_data["val"] = [val_predictions_base, val_references_base, val_inst_base]
inferenced_data["test"] = [test_predictions, test_references, test_inst]

store(inferenced_data, "./fine-tuned-rag-inference.json")

val_results = evaluate(val_predictions_base, val_references_base, metrics=["rouge", "bleu"])
print("Validation Rouge results:", val_results["rouge"])
print("Validation BLEU results:", val_results["bleu"])

test_results = evaluate(test_predictions, test_references, metrics=["rouge", "bleu"])
print("Test Rouge results:", test_results["rouge"])
print("Test BLEU results:", test_results["bleu"])

NameError: name 'rouge' is not defined

In [ ]:
FastLanguageModel.for_inference(model)
query = "Suggest a healthy pasta recipe under 400 calories."

# Step 1: retrieve relevant context
retrieved_nodes = retriever.retrieve(query)
context_texts = "\n\n".join([n.node.get_content() for n in retrieved_nodes])

# Step 2: construct augmented prompt
rag_prompt = f"""You are a nutrition expert.
Here are some reference documents:
{context_texts}

Question: {query}

Answer clearly and concisely:
"""

# Step 3: generate the final answer using your local model
answer = model.generate(rag_prompt)
print(answer)


AttributeError: 'str' object has no attribute 'shape'

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = index.as_retriever(similarity_top_k=5)
query_engine = RetrieverQueryEngine.from_args(retriever=retriever)

query_str = "Suggest a healthy pasta recipe under 400 calories."
response = query_engine.query(query_str)
print(response)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
random_state=3407

# Load the model and tokenizer from the Hugging Face Hub
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "azimidokht/recipe-recom-fine-tuned",  # your HF repo
    max_seq_length = 2048,  # or the same you used during training
    dtype = None,           # or "float16"/"bfloat16" depending on your GPU
    load_in_4bit = True,    # set to True if you trained/quantized with 4-bit
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = random_state,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
prompt = """
### Human:
{}

### Assistant
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("azimidokht/recipe-recom", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

split_dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=random_state)

train_dataset = split_dataset["train"]
temp_dataset  = split_dataset["test"]

val_test_split = temp_dataset.train_test_split(test_size=0.5, shuffle=True, seed=random_state)

val_dataset  = val_test_split["train"]
test_dataset = val_test_split["test"]


In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
llm = HuggingFaceLLM(
  model_name = "azimidokht/recipe-recom-fine-tuned",  # your HF repo
    max_seq_length = 2048,  # or the same you used during training
    dtype = None,           # or "float16"/"bfloat16" depending on your GPU
    load_in_4bit = True,
)

In [ ]:
import pandas as pd
from pathlib import Path

input_path = Path("/content/drive/MyDrive/Colab Notebooks/dataset/pp_recipes.csv")
output_dir = Path("/content/drive/MyDrive/Colab Notebooks/dataset/splits")
output_dir.mkdir(exist_ok=True)

chunksize = 50_000  # adjust depending on available RAM
for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
    chunk.to_csv(output_dir / f"part_{i}.csv", index=False)


/tmp/ipython-input-2893009431.py:9: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.7 MB/s e

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
import chromadb

chroma_path = "/content/chroma_storage"
db = chromadb.PersistentClient(path=chroma_path)
chroma_collection = db.get_or_create_collection("dataset_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = embed_model
Settings.transformations = [SentenceSplitter(chunk_size=1024)]

storage_dir = Path("/content/index_storage")
storage_context = StorageContext.from_defaults(persist_dir=storage_dir)

for csv_path in Path("/content/drive/MyDrive/Colab Notebooks/dataset/splits").glob("*.csv"):
    documents = SimpleDirectoryReader(input_files=[str(csv_path)]).load_data()
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
    storage_context.persist()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/index_storage/docstore.json'

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="/content/index_storage")
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()

query_str = "Suggest a healthy pasta recipe under 400 calories."
response = query_engine.query(query_str)
print(response)


In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_str = "Suggest a healthy pasta recipe under 400 calories."

query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))